In [18]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(2, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
 
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 19s 319us/step - loss: 0.4923 - acc: 0.8513 - val_loss: 0.1755 - val_acc: 0.9472
Epoch 2/12
60000/60000 [==============================] - 18s 305us/step - loss: 0.2261 - acc: 0.9339 - val_loss: 0.1271 - val_acc: 0.9612
Epoch 3/12
60000/60000 [==============================] - 19s 319us/step - loss: 0.1852 - acc: 0.9454 - val_loss: 0.1094 - val_acc: 0.9669
Epoch 4/12
60000/60000 [==============================] - 21s 355us/step - loss: 0.1658 - acc: 0.9516 - val_loss: 0.0999 - val_acc: 0.9686
Epoch 5/12
60000/60000 [==============================] - 19s 312us/step - loss: 0.1514 - acc: 0.9556 - val_loss: 0.0884 - val_acc: 0.9725
Epoch 6/12
60000/60000 [==============================] - 19s 310us/step - loss: 0.1408 - acc: 0.9585 - val_loss: 0.0846 - val_acc: 0.9738
Epoch 7/12
60000/60000 [===================

In [21]:
model.save('my_model_v1.h5')

In [32]:
from keras.models import Model
import numpy as np
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer('dense_3').output)
intermediate_output = intermediate_layer_model.predict(x_test)


In [12]:
# the way to access intermediate output


intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer('flatten_1').output)
intermediate_output = intermediate_layer_model.predict(x_test)
print(intermediate_output.shape)
#for i in range(338):
   # if intermediate_output[0][i]!=-0.0 or intermediate_output[0][i]>0:
          #  print(i,intermediate_output[0][i])
t = 0
j = 1
for i in range(338):
    t += model.layers[3].get_weights()[0][i][j]*intermediate_output[0][i]
print(t+model.layers[3].get_weights()[1][j])

(10000, 338)
0.5483485836084583


In [30]:
# write trained data into file

k1_file = open('kernel1.txt','w')
k2_file = open('kernel2.txt','w')

for i in range(3):
    for j in range(3):
        k1_file.write(str(model.layers[0].get_weights()[0][i][j][0][0])+'\n')
        k2_file.write(str(model.layers[0].get_weights()[0][i][j][0][1])+'\n')
        
k1_file.close()
k2_file.close()

b1_file = open('bias1.txt','w')
b2_file = open('bias2.txt','w')

b1_file.write(str(model.layers[0].get_weights()[1][0]))
b2_file.write(str(model.layers[0].get_weights()[1][1]))
b1_file.close()
b2_file.close()

In [19]:
# each layer configuration

for lay in model.layers:
    print(lay.name)
    print(lay.get_weights())

conv2d_2
[array([[[[ 0.44204316,  0.22141126]],

        [[ 0.20372748,  0.8207506 ]],

        [[ 0.32394782,  0.9904973 ]]],


       [[[ 0.08491429, -0.10046753]],

        [[ 0.66045576,  0.36949417]],

        [[ 0.35053986,  0.553357  ]]],


       [[[ 0.5027545 , -1.3718123 ]],

        [[ 0.50884897, -1.3769588 ]],

        [[-0.13011555, -1.5518826 ]]]], dtype=float32), array([-0.00734525, -0.05760591], dtype=float32)]
max_pooling2d_2
[]
flatten_2
[]
dense_3
[array([[-0.02053294, -0.0588712 , -0.00654707, ..., -0.06022764,
         0.03747564, -0.09429256],
       [-0.057024  , -0.09043556, -0.07250241, ...,  0.00424495,
         0.09357055, -0.044453  ],
       [-0.00714752, -0.00416151,  0.05735605, ...,  0.07807866,
        -0.07901908, -0.03027713],
       ...,
       [ 0.02439989, -0.00335889,  0.07014727, ...,  0.09435672,
         0.02215195, -0.08084667],
       [-0.06409877,  0.06748356, -0.05357154, ..., -0.02832441,
         0.06833386,  0.0126288 ],
       [-0.0863

In [20]:
# write dense kernel into file

dk1_f = open('dk1.txt','w')
for i in range(338):
    for j in range(128):
        dk1_f.write(str(model.layers[3].get_weights()[0][i][j])+'\n')
dk1_f.close()

dk2_f = open('dk2.txt','w')
for i in range(128):
    for j in range(10):
        dk2_f.write(str(model.layers[5].get_weights()[0][i][j])+'\n')
dk2_f.close()

db1_f = open('db1.txt','w')
for i in range(128):
    db1_f.write(str(model.layers[3].get_weights()[1][i])+'\n')
db1_f.close()

db2_f = open('db2.txt','w')
for i in range(10):
    db2_f.write(str(model.layers[5].get_weights()[1][i])+'\n')
db2_f.close()